In [381]:
import pandas as pd
import os
import numpy as np

In [382]:
directory  = os.getenv("HOME") + '/aiffel/aiffelthon/meta'

In [383]:
os.listdir(directory)

['reg_1.json',
 'meta_data_included_raw_v2.csv',
 '.ipynb_checkpoints',
 'meta_eng_raw_0306_v1.csv',
 'kor_text.xlsx',
 'reg_3.json',
 'meta_data_included_raw.csv',
 'meta_raw_eng_corrected_sts_v1.csv',
 'meta_data_raw_eng_match_6m.csv',
 'kor_text.txt',
 'meta_raw_eng_sts_v1.csv',
 'reg_4.json',
 'meta_raw_eng_corrected_sts_v1_needs_correction_translated.xlsx',
 '사투리화_데이터셋_0309.csv',
 'meta_raw_eng_corrected_sts_v1_needs_correction.xlsx',
 'kw_t.json',
 'reg_5.json',
 'meta_raw_eng_corrected_sts_v_final.csv',
 'meta_data_raw_0306.csv',
 'reg_2.json']

In [384]:
df = pd.read_csv(directory + '/meta_raw_eng_corrected_sts_v_final.csv')

In [385]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 949888 entries, 0 to 949887
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   949888 non-null  int64  
 1   reg          949888 non-null  object 
 2   topic        949888 non-null  object 
 3   stdn         949888 non-null  object 
 4   dial         949888 non-null  object 
 5   eng          949888 non-null  object 
 6   eojeol_stdn  949888 non-null  object 
 7   sts          949888 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 58.0+ MB


In [386]:
del df['Unnamed: 0']

In [387]:
regs = df.reg.unique().tolist()
jd_e,jj_e, cc_e, gs_e, kw_e = dict(),dict(),dict(),dict(),dict()
eojs = [jd_e,jj_e, cc_e, gs_e, kw_e]

In [388]:
for reg, eoj in zip(regs, eojs) :
    eoj['eojels'] = df.loc[df['reg'] == reg,'eojeol_stdn'].values

In [389]:
import re

pairs = {}

for reg, eoj in zip(regs,eojs) :
    pairs[reg] = {}
    for i in eoj['eojels'] :
        match = re.search(r'(?<=\()(\w*)(?=\))',i)
        tg_match = re.search(r'(?<=\/\()(\w*)(?=\))',i)
        if match and tg_match:
            dial = re.search(r'(?<=\()(\w*)(?=\))',i).group()
            src = re.search(r'(?<=\/\()(\w*)(?=\))',i).group()
            pairs[reg].update({src : dial})

In [390]:
for reg, eoj in zip(regs,eojs) :
    temp = df.loc[df['reg'] == reg]
    temp_dial = temp.dial.values
    temp_stdn = temp.stdn.values
    test_dial_v = []
    temp_reg = [reg for _ in range(len(temp))]

    for sentence in temp_dial :
        new = []
        for x in sentence.split() :
            try :
                new.append(pairs[reg][x])
            except :
                new.append(x)
        test_dial_v.append(' '.join(new))
    df.loc[df['reg'] == reg,'dial_edit_f']= test_dial_v

In [391]:
#시소러스 불러오기
with open('eojeol_subs.json','r') as f :
    tjv = json.load(f)

In [392]:
for reg in regs :
    with open('test.txt','w') as f:
        for i in df[df['reg'] == reg].dial_edit_f.values :
            f.write(i +' \n')

    with open('test.txt','r') as f:
        raw = f.read()

    for rx, sb in tjv[reg] :
        raw = re.sub("{}".format(rx),sb,raw)

    df.loc[df['reg'] == reg,'dial_edit_f2'] = raw.splitlines()

In [393]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 949888 entries, 0 to 949887
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   reg           949888 non-null  object 
 1   topic         949888 non-null  object 
 2   stdn          949888 non-null  object 
 3   dial          949888 non-null  object 
 4   eng           949888 non-null  object 
 5   eojeol_stdn   949888 non-null  object 
 6   sts           949888 non-null  float64
 7   dial_edit_f   949888 non-null  object 
 8   dial_edit_f2  949888 non-null  object 
dtypes: float64(1), object(8)
memory usage: 65.2+ MB


In [395]:
df.to_csv(directory + '/meta_raw_eng_corrected_sts_saturization_v_final.csv')

In [357]:
df_t = df[['reg','stdn','dial','dial_edit_f','dial_edit_f2']].copy()

In [358]:
export_df = pd.DataFrame(columns=['reg','stdn','dial','dial_edit_f','dial_edit_f2'])
for reg in regs :
    temp = df_t.loc[df_t['reg'] == reg].sample(100)
    export_df = pd.concat([export_df,temp],axis=0)

In [359]:
# export_df['reg'] =export_df['reg'].map({'jd':'전라도','jj':'제주도','gs':'경상도','kw':'강원도','cc':'충청도'})

In [360]:
export_df.head(30)

,reg,stdn,dial,dial_edit_f,dial_edit_f2
3143,jd,이런 생각이 또 드는 거야 근데 아무렴 봐도 너무 깔끔하게 잘 들고 다닐 거 같애가지고,이런 생각이 또 드는 거야 근데 암만 봐도 너무 깔끔하게 잘 들고 다닐 거 같애가지고,이른 생각이 또잉 드는 것이여 근디 암만 봐도잉 하도 깰끔하게 달 들고잉 댕길 거이...,이른 생각이 또잉 드는 것이여 근디 암만 봐도잉 하도 깰끔하게 달 들고잉 댕길 거이...
37206,jd,아야 너 쪼끔 너무 왜소해 보이니까 그만 빼도 되겠다 좀 왜소해 보이면 것 그런,아야 너 쪼끔 너무 왜소해 보이니까 그만 빼도 되겄다 쫌 왜소해 보이면 것 그런,아야 니 쪼까 하도 왜소해 보이니껩 고만 빼도 되겠다 좀 왜소해 보이믄 꺼 그른,아야 니 쪼까 하도 왜소해 보이니껩 고만 빼도 되겄다 좀 왜소해 보이믄 꺼 그른
29901,jd,진짜 뭔가 그~ 프리미엄 버스 한 번 타고 나니까 그냥 일반 버스를 못 타고 다니겠어.,진짜 뭔가 그~ 프리미엄 버스 한 번 타고 나니까 그냥 일반 버스를 못 타고 다니겄어.,지짜 먼가 그~ 프리미엄 뻐쓰 헌 분 타고잉 난께 걍 일반 뻐스를 모 타고잉 다니겄어.,지짜 먼가 그~ 프리미엄 뻐쓰 헌 분 타고잉 난께 걍 일반 뻐스를 모 타고잉 다니겄어.
34711,jd,이랬더니 그럼 목포로 갈까?,이랬더니 그럼 목포로 가까?,이랬드니 금 목포로 가까?,이랬드니 금 목포로 가까?
49815,jd,그거 처음에 올라가면 진짜 죽어 게다가 좀 뾰족한 거더라고 뾰족한 게 중간중간 있더...,그거 처음에 올라가면 진짜 죽어 게다가 좀 뾰족한 거더라고 뾰족한 게 중간중간 있더...,고거 첨에 올라가믄 지짜 뒤져 거따가 쫌 뾰조간 거더라고잉 뾰조간 게잉 중간중간 있...,고거 첨에 올라가믄 지짜 뒤져 거따가 쫌 뾰조간 거더라고잉 뾰조간 게잉 중간중간 있...
789395,jd,그 뭔지 알지 하씨 내가 이런걸 보고 온다고 하는데 울어 그런 영화 이제 그런 것이지.,그 뭔지 알제 하씨 내가 이런걸 보고 온다고야 한디 울어 그런 영화 이제 그런 것이제.,고 먼지 알제 하씨 나가 이런걸 봐보고 온다고야 한디 울어 그른 영화잉 인제 그른 ...,고 먼지 알제 하씨 나가 이런걸 봐보고 온다고야 한디 울어 그른 영화잉 인제 그른 ...
58463,jd,그래서 그 남자애가 또 심지어 약간 조금 풋풋하게 귀엽게 생겼잖아?,그래서 그 남자애가 또 심지어 약간 쫌 풋풋하게 귀엽게 생겼잖아?,긍께 고 머시매가 또잉 심지어 약깐 좀 풋풋하게 귀엽게 생겼잖아?,긍께 고 머시매가 또잉 심지어 약깐 좀 풋풋하게 귀엽게 생겼잖아?
17887,jd,응 그러니까 피할 수 없는 그런 거 같애 누구나 사실,응 그니까 피할 수 없는 그런 거 같애 누구나 사실,잉 긍게 피할 수 음는 그른 거이 같애잉 누구나 실은,잉 긍게 피할 수 음는 그른 거이 같애잉 누구나 실은
90125,jd,그래가지고 우리 아버지 얼굴도 모르는 아버지가,그래가꼬 우리 아부지 얼굴도 모르는 아버지가,가꼬 울 아부지 얼굴도 모른 아부지가,가꼬 울 아부지 얼굴도 모른 아부지가
238586,jd,이 사람 같은 사람은 조금 내 이상형이 아니다 내 이상형과 거리가 멀다 이렇게 생각...,이 사람 같은 사람은 쫌 내 이상형이 아니다 내 이상형과 거리가 멀다 이렇게 생각 ...,요 사람 깥은 사람은 좀 내잉 이상형이 아이다 내잉 이상형과 꺼리가 멀다 이르케 생...,요잉 사람 깥은 사람은 좀 내잉 이상형이 아이다 내잉 이상형과 꺼리가 멀다 이르케 ...


In [165]:
export_df.to_csv(directory + '/사투리화_데이터셋_0309.csv', index=False)

### W2Vec replacement

In [167]:
directory = os.getenv("HOME") +'/aiffel/aiffelthon'

In [168]:
os.listdir(directory)

['MODELtf_ckpts_gd12_r1',
 'reg_2',
 'meta',
 'w2v_meaning_jj.model',
 'past_files',
 'onnx',
 '.ipynb_checkpoints',
 '[라벨]제주도_학습용데이터_1.zip',
 'corpus_ko.txt',
 '[라벨]전라도_학습데이터_1.zip',
 'added',
 'final',
 '[라벨]경상도_학습데이터_1.zip',
 'reg_1',
 'onnx_sts',
 'reg_4',
 'spm',
 'datset_full_raw_needs_translation_0221_v2.csv',
 '[라벨]충청도_학습데이터_1.zip',
 'reg_5',
 'saturi_crawling_thesaurus_0223.csv',
 'final_dataset.zip',
 '[라벨]강원도_학습데이터_1.zip',
 'reg_3',
 'DATA',
 'corpus_en.txt']

In [363]:
saturi_data = pd.read_csv(directory + '/saturi_crawling_thesaurus_0227.csv')

In [364]:
saturi_data.head()

,dial,original,meaning,reg
0,분쉬,분수,"분수의 방언, 사물을 분별하는 지혜.",jj
1,분시,분수,"분수의 방언, 사물을 분별하는 지혜.",jj
2,분절,분결,"분결의 방언, 분한 마음이 왈칵 일어난 바람.",jj
3,분채,부채,"부채의 방언, 손으로 흔들어 바람을 일으키는 물건. 대오리로 살을 하고 종이나 헝...",jj
4,분채질,부채질,"부채질의 방언, 부채를 흔들어 바람을 일으키는 일.",jj


In [365]:
print(len(saturi_data))

78042


In [172]:
print(saturi_data.isnull().sum())

dial        0
original    0
meaning     0
reg         0
dtype: int64


In [244]:
### JJ_data

jj = saturi_data[saturi_data['reg'] == 'jj'].copy()

print(len(jj))

14622


In [245]:
import gensim 

import numpy as np

jj['word2vec'] = jj['dial'] + '' + jj['original']
jj_sample = jj['word2vec'].apply(gensim.utils.simple_preprocess) *3# series 이용

jj_sample

0              [분쉬, 분수, 분쉬, 분수, 분쉬, 분수]
1              [분시, 분수, 분시, 분수, 분시, 분수]
2              [분절, 분결, 분절, 분결, 분절, 분결]
3              [분채, 부채, 분채, 부채, 분채, 부채]
4        [분채질, 부채질, 분채질, 부채질, 분채질, 부채질]
                      ...              
14617          [야답, 여덟, 야답, 여덟, 야답, 여덟]
14618          [마은, 마흔, 마은, 마흔, 마은, 마흔]
14619          [설룬, 서른, 설룬, 서른, 설룬, 서른]
14620          [설흔, 서른, 설흔, 서른, 설흔, 서른]
14621    [대요슷, 대여섯, 대요슷, 대여섯, 대요슷, 대여섯]
Name: word2vec, Length: 14622, dtype: object

In [247]:
jj.loc[(jj['original'].str.contains('아이'))]

,dial,original,meaning,reg,word2vec
217,사나의놈,사내아이,"사내아이의 방언 제주 지역에서는 나의놈으로도 적는다., 남자아이를 친근하게 이르...",jj,사나의놈 사내아이
766,햇아이,어린아이,"어린아이의 방언, 나이가 적은 아이.",jj,햇아이 어린아이
2107,기집아의,계집아이,"계집아이의 방언, 여자아이를 낮잡아 이르는 말.",jj,기집아의 계집아이
2108,기집아이,계집아이,"계집아이의 방언, 여자아이를 낮잡아 이르는 말.",jj,기집아이 계집아이
3045,곤아기,갓난아이,"갓난아이의 방언, 태어난 지 얼마 되지 아니한 아이.",jj,곤아기 갓난아이
3046,곤애기,갓난아이,"갓난아이의 방언, 태어난 지 얼마 되지 아니한 아이.",jj,곤애기 갓난아이
3225,어린아기,어린아이,"어린아이의 방언, 나이가 적은 아이.",jj,어린아기 어린아이
3226,어린애기,어린아이,"어린아이의 방언, 나이가 적은 아이.",jj,어린애기 어린아이
3305,얼아이,어린아이,"어린아이의 방언, 나이가 적은 아이.",jj,얼아이 어린아이
3702,두리아기,어린아이,"어린아이의 방언, 나이가 적은 아이.",jj,두리아기 어린아이


In [174]:
jj['word2vec'] = jj['dial'] + '' + jj['original']
jj_sample = jj['word2vec'].apply(gensim.utils.simple_preprocess) *3# series 이용

In [175]:
jj_sample

0              [분쉬, 분수, 분쉬, 분수, 분쉬, 분수]
1              [분시, 분수, 분시, 분수, 분시, 분수]
2              [분절, 분결, 분절, 분결, 분절, 분결]
3              [분채, 부채, 분채, 부채, 분채, 부채]
4        [분채질, 부채질, 분채질, 부채질, 분채질, 부채질]
                      ...              
14617          [야답, 여덟, 야답, 여덟, 야답, 여덟]
14618          [마은, 마흔, 마은, 마흔, 마은, 마흔]
14619          [설룬, 서른, 설룬, 서른, 설룬, 서른]
14620          [설흔, 서른, 설흔, 서른, 설흔, 서른]
14621    [대요슷, 대여섯, 대요슷, 대여섯, 대요슷, 대여섯]
Name: word2vec, Length: 14622, dtype: object

In [218]:
from gensim.models import Word2Vec

def w2v_train(data, name) :
    model = Word2Vec(window = 4, min_count = 3, workers = 4, sg = 1, epochs = 50)
    model.build_vocab(data)
    model.train(data, total_examples=model.corpus_count, epochs = model.epochs)
    model.save(directory +f'/w2v_meaning_{name}.model')  
    
    return model

In [219]:
model_jj = w2v_train(jj_sample,'jj')
model_jj.wv.most_similar('분수')

[('분쉬', 0.9933437705039978),
 ('분시', 0.9930066466331482),
 ('가랍다', 0.6450494527816772),
 ('가룹다', 0.6360270977020264),
 ('가룝다', 0.6340727210044861),
 ('가렵다', 0.622137188911438),
 ('똑기', 0.6180091500282288),
 ('동경이', 0.6156563758850098),
 ('목고디', 0.6146266460418701),
 ('야가지', 0.6144046783447266)]

In [248]:
model_jj.wv.most_similar('어린아이')

[('두린아이', 0.9944793581962585),
 ('두리아기', 0.9941440224647522),
 ('어린아기', 0.9939889311790466),
 ('얼아이', 0.9937254190444946),
 ('두린애기', 0.9935956001281738),
 ('어린애기', 0.9932720065116882),
 ('햇아이', 0.9928451180458069),
 ('베뿌기', 0.6241863369941711),
 ('졸곡', 0.6202628016471863),
 ('살쩍밀이', 0.6171629428863525)]

In [186]:
import random

In [334]:
from konlpy.tag import Mecab
mecab = Mecab()
def eda_sr(sentence, word2vec) :
    words = mecab.pos(sentence)
    sentence_new = sentence
    for i,pos in words :
        rand_num =random.random()
        if pos == 'NNG':
            try :
                if 0.3>rand_num > 0 :
                    new = word2vec.wv.most_similar(i)[0][0]
                elif rand_num > 0.6 :                
                    new = word2vec.wv.most_similar(i)[2][0]
                else :
                    new = word2vec.wv.most_similar(i)[1][0]
                sentence_new = re.sub(i,new,sentence_new)
            except :
                continue
        else :
            continue
    return sentence_new

In [352]:
original = '어린아이 !'
eda_sr(original, model_jj)

'두린아이 !'

In [375]:
import tqdm

def change_expression(data,text_file, reg) :
    data = data[data['reg'] == reg].copy()
#     data['original'] = data['meaning'].apply(lambda x : x.split('의 방언')[0] if '의 방언' in x else np.nan)
#     data.dropna(subset=['original'], inplace=True)
    data['word2vec'] = data['dial'] + '' + data['original']
    data_sample = data['word2vec'].apply(gensim.utils.simple_preprocess) *3# series 이용
    model = w2v_train(data_sample, reg)
    
    text_files = text_file[text_file['reg'] == reg].copy()
    text_files = text_files['dial_edit_f2'].values # substitute data
    new_sentence = []
    for i in tqdm.tqdm(range(len(text_files))) :
        new = eda_sr(text_files[i], model)
        new_sentence.append(new)
    return new_sentence

In [376]:
saturi_data

,dial,original,meaning,reg
0,분쉬,분수,"분수의 방언, 사물을 분별하는 지혜.",jj
1,분시,분수,"분수의 방언, 사물을 분별하는 지혜.",jj
2,분절,분결,"분결의 방언, 분한 마음이 왈칵 일어난 바람.",jj
3,분채,부채,"부채의 방언, 손으로 흔들어 바람을 일으키는 물건. 대오리로 살을 하고 종이나 헝...",jj
4,분채질,부채질,"부채질의 방언, 부채를 흔들어 바람을 일으키는 일.",jj
...,...,...,...,...
78037,달싹허다,반반하다,"반반하다의 방언, 구김살이나 울퉁불퉁한 데가 없이 고르고 반듯하다.",cc
78038,대근하다,고단하다,"고단하다의 방언, 몸이 지쳐서 느른하다.",cc
78039,대무하다,크다,"크다의 방언, 사람이나 사물의 외형적 길이, 넓이, 높이, 부피 따위가 보통 정도...",cc
78040,대장실하다,담차다,"담차다의 방언, 겁이 없이 대담하고 여무지다.",cc


In [377]:
for reg in regs :
    dial_new_list = change_expression(saturi_data,export_df, reg)
    export_df.loc[df['reg'] == reg,'dial_edit_f3'] = dial_new_list

100%|██████████| 100/100 [00:00<00:00, 1312.01it/s]


In [379]:
export_df.loc[df['reg'] == reg].sample(10)

,reg,stdn,dial,dial_edit_f,dial_edit_f2,dial_edit_f3
188089,kw,엄마는 지금 오빠에 대해서 말하는 게 아니란 말이야.,엄마는 지꿈 오빠에 대해서 말하는 게 아이란 말이나.,엄니는 지꿈 오빠에 대해서 말하는 기 아이란 말이나.,엄니는 지꿈 오빠에 대해서 말하는 기 아이란 말이나.,엄니는 지꿈 오빠에 대해서 말하는 기 토리란 말이나.
443547,kw,사모님 제비 한마리 키우십쇼.,사모님 제미 한마리 키우십쇼.,사무님 지기미 한바리 키우십쇼.,사무님 지기미 한바리 키우십쇼.,너부리님 지기미 한몸약 키우십쇼.
308147,kw,저희 집엔 어떻게?,즈네 집엔 어뜨케?,즈네 집엔 어뜨케?,즈네 집엔 어뜨케?,즈네 집엔 어뜨케?
483383,kw,음료를 꼭 같이 먹어 왜냐면 그 쓴맛이 조금 싫어 가지고,음료를 꼭 같이 먹어 왜냐면 그 쓴맛이 쫌 싫어 가지고,음료를 꽉 매치루 먹나 왜냐먼 그이 쓴맛이 좀 싫아 가주구,음료를 꽉 매치루 먹나 왜냐먼 그이 쓴맛이 좀 싫아 가주구,음료를 꽉 매치루 먹나 왜냐먼 그이 쓴맛이 좀 싫아 가주구
299988,kw,음료수를 지금 마저 마셔요.,음료수를 지굼 마주 마셔요.,음료수를 지굼 맞조이 마셔요.,음료수를 지굼 맞조이 마셔요.,음료수를 지굼 맞조이 마셔요.
424212,kw,그것은 사리에 맞지 않어.,그근 사리에 맞지 않어.,그근 사리에 맞제 않어.,그근 사리에 맞제 않어.,그근 사리에 맞제 않어.
547778,kw,참 아유 여우짓 하는데 엄마까지 넘어 갔어.,참 아유 여깽이짓 하는데 엄마까지 넘어 갔어.,참말애 아유 여깽이짓 카는데 어머이까지 늠에 갔어.,참말애 아유 여깽이짓 카는데 어머이까지 늠에 갔어.,참말애 야야라이 영우짓 카는데 어멍이까지 늠에 갔어.
456844,kw,어머님이 동서 연락 안된다고 나한테까지 전화하셨는데?,어머님이 동세 연락 안된다고 나한테까지 즌화하셨는데?,어머이이 동세 기별 안된다고 나한테까지 즌화하셨는데?,어머이이 동세 기별 안된다고 나한테까지 즌화하셨는데?,어머이이 동서 기별 안된다고 나한테까지 즌화하셨는데?
400331,kw,그 아줌마 오면 사과는 하는거다?,그 아줌마 오면 사개는 하는거다?,그이 아주머이 오문 사개는 하는거다?,그이 아주머이 오문 사개는 하는거다?,그이 아재 오문 사괴는 하는거다?
514879,kw,점심 먹었어요.,점슴 먹었어요.,점슴 먹었어요.,점슴 먹었사.,즘숨 먹었사.


In [372]:
len(dial_new_list)

500

In [ ]:
export_df.head(30)

---

In [67]:
# 충청도
cc_rep =[('(?<=)(에)(?= )','이'),
('(?<=)(한테)(?= )','헌티'),
('(?<=)(보고)(?= )','헌티'),
('(?<=)(더러)(?= )','헌티'),
('(?<=)(만치)(?= )','맨치'),
('(?<=)(만큼)(?= )','맨큼'),
('(?<=)(밖에)(?= )','밲이'),
('(?<=)(부터)(?= )','부통'),
('(?<=)(까지)(?= )','까정'),
('(?<=)(와)(?= )','랑'),
('(?<=)(으로)(?= )','을루'),
('(?<=)(거로)(?= )','걸루'),
('(?<=)(대\?)(?= )','다냐?'),
('(?<=)(아서)(?= )','아가주구'),
('(?<=)(을까봐|으까봐)(?= )','을깨미'),
('(?<=)(가봐)(?= )','가벼'),
('(?<=)(는가보네)(?= )','는개비네'),
('(?<=)(거든)(?= )','걸랑'),
('(?<=)(길래)(?= )','걸래'),
('(?<=)(예요)(?=\.| )','유'),
('(?<=)(세요)(?=\.| )','슈'),
('(?<=)(요)(?=\.| )','유'),
('(?<=)(죠)(?=\.| )','지유'),
('(?<=)(거예요)(?=\.| )','뀨'),
('(?<=)(거야)(?=\.|\?| )','겨'),
('(?<=)(야)(?= )','여'),
('(?<=)(한거야)(?= )','핸겨'),
('(?<=)(걸)(?= )','껴'),
('(?<=)(래)(?=\?| )','텨?'),
('(?<=)(해)(?=\.| )','혀'),
('(?<=)(해야지)(?= )','하야지'),
('(?<=)(했어)(?= )','혔어'),
('(?<=)(대)(?= )','댜'),
('(?<=)(는데)(?= )','는디'),
('(?<=)(돼)(?= )','댜'),
('(?<=)(니까|으니까)(?= )','니께'),
('(?<=)(으게)(?= )','으께'),
('(?<=)(더니)(?= )','드니'),
('(?<=)(스레)(?= )','시리'),
('(?<=)(더라)(?= )','드라'),
('(?<=)(고)(?= )','구'),
('(?<=)(려고)(?= )','라구'),
('(?<=)(지도)(?= )','던'),
('(?<=)(겠지)(?= )','겄지'),
('(?<=)(려나)(?= )','란가'),
('(?<=)(있냐\?)(?= )','있간'),
('(?<=)(니)(?=\?| )','냐'),
('(?<=)(야)(?=\?| )','냐'),
('(?<=)(쪽)(?= )','짝')]

In [68]:
##전라
jd_rep = [
['(?<=)(는데)(?= )' ,'간디'],
['(?<=)(인가보다|인가보네)(?= )' ,'갑네'],
['(?<=)(겠다)(?= )' ,'겄다'],
['(?<=)(셔요)(?=\.| )' ,'게요'],
['(?<=)(고 싶다)(?=\.| )' ,'고 잡다'],
['(?<=)(갈까)(?= )' ,'끄나'],
['(?<=)(보니까)(?= )' ,'본께'],
['(?<=)(하니까)(?= )' ,'한께'],
['(?<=)(대니)(?= )' ,'다냐'],
['(?<=)(다니까)(?= )' ,'당께'],
['(?<=)(라니까)(?= )' ,'랑께'],
['(?<=)(듯이)(?= )' ,'데끼'],
['(?<=)(더라)(?= )' ,'드라고'],
['(?<=)(데)(?= )' ,'디'],
['(?<=)(대)(?= )' ,'디야'],
['(?<=)(보려고)(?= )' ,'볼라고'],
['(?<=)(하려고)(?= )' ,'할라고'],
['(?<=)(는지)(?= )' ,'랑가'],
['(?<=)(럽다)(?= )' ,'룹다'],
['(?<=)(려워)(?= )' ,'루와'],
['(?<=)(려워)(?= )' ,'루와잉'],
['(?<=)(처럼)(?= )' ,'멘치'],
['(?<=)(면서)(?= )' ,'시로'],
['(?<=)(버려)(?= )' ,'부러'],
['(?<=)(버리지)(?= )' ,'블제'],
['(?<=)(소)(?= )' ,'쇼'],
['(?<=)(겠다)(?=\.| )' ,'겄다'],
['(?<=)(대다)(?= )' ,'쌓다'],
['(?<=)(요)(?=\.| )' ,'요잉'],
['(?<=)(지요)(?=\.| )' ,'지라']
]

In [105]:
#경상도

gs_rep = [
['(?<=)(더냐)(?=\.|\?| )' , '다~아'],
['(?<=)(런가)(?=\.|\?| )' , '란고'],
['(?<=)(에게)(?=\.| )' , '한테'],
['(?<=)(만치)(?=\.| )' , '매치'],
['(?<=)(만큼)(?=\.| )' , '맨큼'],
['(?<=)(처럼)(?=\.| )' , '맨치로'],
['(?<=)(만큼도)(?=\.| )' , '맨도'],
['(?<=)(는)(?=\.| )' , '느'],
['(?<=)(밖에)(?=\.| )' , '밲이'],
['(?<=)(까지)(?=\.| )' , '까증'],
['(?<=)(에야)(?=\.| )' , '이사'],
['(?<=)(와)(?=\.| )' , '랑'],
['(?<=)(면)(?=\.| )' , '머'],
['(?<=)(면서)(?=\.| )' , '민시'],
['(?<=)(아서)(?=\.| )' , '아가'],
['(?<=)(든지)(?=\.| )' , '든둥'],
['(?<=)(다)(?=\.| )' , '대이'],
['(?<=)(라)(?=\.| )' , '래이'],
['(?<=)(자)(?=\.| )' , '대이'],
['(?<=)(이야)(?=\.| )' , '이라'],
['(?<=)(이예요)(?=\.| )' , '이라예'],
['(?<=)(이었다)(?=\.| )' , '이랐다'],
['(?<=)(니다)(?=\.| )' , '니더'],
['(?<=)(니까)(?=\.| )' , '니께'],
['(?<=)(게)(?=\.| )' , '께'],
['(?<=)(더니만)(?=\.| )' , '디마는'],
['(?<=)(지마는)(?=\.| )' , '아도'],
['(?<=)(거니와)(?=\.| )' , '지마는'],
['(?<=)(게)(?=\.| )' , '구로'],
['(?<=)(스레)(?=\.| )' , '스리'],
['(?<=)(더라)(?=\.| )' , '대'],
['(?<=)(고)(?=\.| )' , '꼬'],
['(?<=)(느라)(?=\.| )' , '니라'],
['(?<=)(다느니)(?=\.| )' , '다니이'],
['(?<=)(다너니)(?=\.| )' ,'다디~이'],
['(?<=)(지)(?=\.| )' , '지를'],
['(?<=)(지도)(?=\.| )' , '도'],
['(?<=)(겠)(?=\.| )' , '겄'],
['(?<=)(고는)(?=\.| )' , '골랑'],
['(?<=)(냐)(?=\.|\?| )' , '노'],
['(?<=)(지)(?=\.|\?| )' , '제'],
['(?<=)(까)(?=\.|\?| )' , '꼬'],
['(?<=)(다고)(?=\.| )' , '다카이'],
['(?<=)(어)(?=\.|\?| )' , '노'],
['(?<=)(요)(?=\.| )' , '예'],
['(?<=)(만)(?=\.| )' , '마']
]


In [106]:
#제주도
jj_rep = [
    ['(?<=)(한테)(?=\.| )' , '안티'],
['(?<=)(에게)(?=\.| )' , '신디'],
['(?<=)(에게)(?=\.| )' , '아피'],
['(?<=)(더러)(?=\.| )' , '가라'],
['(?<=)(더러는)(?=\.| )' , '가란'],
['(?<=)(까지)(?=\.| )' , '까장'],
['(?<=)(한들)(?=\.| )' , '한덜'],
['(?<=)(나마)(?=\.| )' , '나따나'],
['(?<=)(으로)(?=\.| )' , '데레'],
['(?<=)(로)(?=\.| )' , '러레'],
['(?<=)(으로는)(?=\.| )' , '데렌'],
['(?<=)(들랑)(?=\.| )' , '덜랑'],
['(?<=)(들도)(?=\.| )' , '덜토'],
['(?<=)(들이)(?=\.| )' , '라'],
['(?<=)(지마는)(?=\.| )' , '라도'],
['(?<=)(이라고)(?=\.| )' , '이란'],
['(?<=)(만큼)(?=\.| )' , '만히'],
['(?<=)(면)(?=\.| )' , '민'],
['(?<=)(부터)(?=\.| )' , '부떠'],
['(?<=)(에게더러)(?=\.| )' , '신더레'],
['(?<=)(한테로)(?=\.| )' , '안트레'],
['(?<=)(한테서)(?=\.| )' , '안티서'],
['(?<=)(한테다가)(?=\.| )' , '안티다'],
['(?<=)(한테랑은)(?=\.| )' , '안티라근에'],
['(?<=)(에는)(?=\.| )' , '애란'],
['(?<=)(에설랑)(?=\.| )' , '에서랑'],
['(?<=)(에며)(?=\.| )' , '에영'],
['(?<=)(을랑은)(?=\.| )' , '으라근'],
['(?<=)(을랑)(?=\.| )' , '으랑'],
['(?<=)(와)(?=\.| )' , '왕'],
['(?<=)(와도)(?=\.| )' , '왕도'],
['(?<=)(와)(?=\.| )' , '왕이랑'],
['(?<=)(처럼)(?=\.| )' , '처름'],
['(?<=)(하고)(?=\.| )' , '하곡'],
['(?<=)(더러)(?=\.| )' , '가라'],
['(?<=)(더러는)(?=\.| )' , '가란'],
['(?<=)(까지)(?=\.| )' , '꼬장'],
['(?<=)(으로)(?=\.| )' , '더레'],
['(?<=)(로)(?=\.| )' , '러레'],
['(?<=)(으로는)(?=\.| )' , '더렌'],
['(?<=)(들도)(?=\.| )' , '덜토'],
['(?<=)(만큼)(?=\.| )' , '만히'],
['(?<=)(부터)(?=\.| )' , '부떠'],
['(?<=)(에게더러)(?=\.| )' , '신더레'],
['(?<=)(한테로)(?=\.| )' , '안트레'],
['(?<=)(한테서)(?=\.| )' , '안티서'],
['(?<=)(한테다가)(?=\.| )' , '안티다'],
['(?<=)(한테랑은)(?=\.| )' , '안티라근에'],
['(?<=)(에는)(?=\.| )' , '에란'],
['(?<=)(에설랑)(?=\.| )' , '에서랑'],
['(?<=)(에며)(?=\.| )' , '에영'],
['(?<=)(을랑은)(?=\.| )' , '으라근'],
['(?<=)(을랑)(?=\.| )' , '으랑'],
['(?<=)(와)(?=\.| )' , '왕'],
['(?<=)(와도)(?=\.| )' , '왕도'],
['(?<=)(처럼)(?=\.| )' , '처름'],
['(?<=)(하고)(?=\.| )' , '하곡']
]

In [107]:
#강원도
kw_rep = [
['(?<=)(어요)(?=\.| )' , '사'],
['(?<=)(었어요)(?=\.| )' , '었드래요'],
['(?<=)(모두)(?=\.| )' , '마카'],
['(?<=)(드러눕다)(?=\.| )' , '둔눠'],
['(?<=)(도)(?=\.| )' , '두'],
['(?<=)(결)(?=\.| )' , '겔'],
['(?<=)(며)(?=\.| )' , '메'],
['(?<=)(벼)(?=\.| )' , '베'],
['(?<=)(켜)(?=\.| )' , '케'],
['(?<=)(펴)(?=\.| )' , '페'],
['(?<=)(혀)(?=\.| )' , '헤'],
['(?<=)(켜)(?=\.| )' , '케'],
['(?<=)(펴)(?=\.| )' , '페'],
['(?<=)(혀)(?=\.| )' , '헤'],
['(?<=)(해)(?=\.| )' , '하라우']]

In [108]:
rep_dict = dict({'cc':cc_rep,'jd':jd_rep,'gs':gs_rep,'jj':jj_rep,'kw':kw_rep})

In [109]:
import json

with open('eojeol_subs.json','w') as f :
    json.dump(rep_dict,f)